In [1]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

In [4]:
os.chdir('/Users/krish/Downloads/')


In [69]:
crimes = pd.read_csv('Crimes2018.csv')
crimes.shape

(1191571, 30)

In [70]:
crimes.dtypes

ID                              int64
Case Number                    object
Date                           object
Block                          object
IUCR                           object
Primary Type                   object
Description                    object
Location Description           object
Arrest                           bool
Domestic                         bool
Beat                            int64
District                        int64
Ward                          float64
Community Area                float64
FBI Code                       object
X Coordinate                  float64
Y Coordinate                  float64
Year                            int64
Updated On                     object
Latitude                      float64
Longitude                     float64
Location                       object
Historical Wards 2003-2015    float64
Zip Codes                     float64
Community Areas               float64
Census Tracts                 float64
Wards       

In [71]:
crimes.isna().sum()

ID                                0
Case Number                       0
Date                              0
Block                             0
IUCR                              0
Primary Type                      0
Description                       0
Location Description           6106
Arrest                            0
Domestic                          0
Beat                              0
District                          0
Ward                             49
Community Area                    2
FBI Code                          0
X Coordinate                  22095
Y Coordinate                  22095
Year                              0
Updated On                        0
Latitude                      22095
Longitude                     22095
Location                      22095
Historical Wards 2003-2015    26012
Zip Codes                     22095
Community Areas               25515
Census Tracts                 25374
Wards                         25507
Boundaries - ZIP Codes      

In [72]:
column_names = list(crimes.columns)
print(column_names)

['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location', 'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Census Tracts', 'Wards', 'Boundaries - ZIP Codes', 'Police Districts', 'Police Beats']


### Dropping Unnecessary Columns

In [73]:
crimes = crimes.drop(columns = ['IUCR', 'FBI Code', 'X Coordinate', 'Y Coordinate','Updated On','Latitude', 'Longitude', 'Location',   
                                'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Census Tracts',
                                'Wards', 'Boundaries - ZIP Codes', 'Police Districts', 'Police Beats' ])

In [74]:
crimes.set_index("ID",inplace=True)


In [76]:
crimes['Date'] = pd.to_datetime(crimes['Date'], format='%m/%d/%Y %I:%M:%S %p')
crimes['date'] = crimes['Date'].dt.date
crimes['time'] = crimes['Date'].dt.time

## Encoding Boolean to Integer 0 and 1

In [90]:
crimes.Arrest = crimes.Arrest.astype("str")
crimes.Arrest = crimes.Arrest.str.replace("False","0")
crimes.Arrest = crimes.Arrest.str.replace("True","1")
crimes.Arrest = crimes.Arrest.astype("int")

In [91]:
crimes.Domestic = crimes.Domestic.astype("str")
crimes.Domestic = crimes.Domestic.str.replace("False","0")
crimes.Domestic = crimes.Domestic.str.replace("True","1")
crimes.Domestic = crimes.Domestic.astype("int")

In [92]:
crimes.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,date,time
ID,,,,,,,,,,,,,,,
13230310,JG447808,2018-01-01,041XX N ASHLAND AVE,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,0,0,1912,19,47.0,6.0,2018,2018-01-01,00:00:00
13235163,JG453401,2018-01-01,027XX N MANGO AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,0,1,2514,25,30.0,19.0,2018,2018-01-01,00:00:00
11848137,JC457303,2018-01-01,081XX S EBERHART AVE,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,RESIDENCE,1,0,631,6,6.0,44.0,2018,2018-01-01,00:00:00
11768997,JC362338,2018-01-01,001XX N ASHLAND AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,RESIDENCE,0,1,1224,12,27.0,28.0,2018,2018-01-01,00:00:00
12057588,JD240441,2018-01-01,048XX N AVERS AVE,OFFENSE INVOLVING CHILDREN,CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,0,1,1712,17,35.0,14.0,2018,2018-01-01,00:00:00


### New column for Day

In [114]:
df = crimes
crimes['Weekday'] = crimes['Date'].dt.weekday

dtype('int64')

In [119]:
crimes.isna().sum()

Case Number                0
Date                       0
Block                      0
Primary Type               0
Description                0
Location Description    6106
Arrest                     0
Domestic                   0
Beat                       0
District                   0
Ward                      49
Community Area             2
Year                       0
date                       0
time                       0
Weekday                    0
dtype: int64

In [124]:
crimes['Primary Type'].unique()

array(['DECEPTIVE PRACTICE', 'OFFENSE INVOLVING CHILDREN',
       'CRIMINAL SEXUAL ASSAULT', 'SEX OFFENSE', 'CRIMINAL DAMAGE',
       'THEFT', 'OTHER OFFENSE', 'MOTOR VEHICLE THEFT', 'BATTERY',
       'ASSAULT', 'CRIM SEXUAL ASSAULT', 'BURGLARY', 'CRIMINAL TRESPASS',
       'OBSCENITY', 'WEAPONS VIOLATION', 'NARCOTICS', 'ROBBERY',
       'LIQUOR LAW VIOLATION', 'HOMICIDE', 'PUBLIC PEACE VIOLATION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'STALKING', 'INTIMIDATION',
       'ARSON', 'HUMAN TRAFFICKING', 'GAMBLING', 'KIDNAPPING',
       'PROSTITUTION', 'NON-CRIMINAL',
       'CONCEALED CARRY LICENSE VIOLATION', 'PUBLIC INDECENCY',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'OTHER NARCOTIC VIOLATION',
       'RITUALISM'], dtype=object)

In [125]:
def translation(value):
    if value in ["CRIMINAL SEXUAL ASSAULT", "CRIMINAL DAMAGE", "HOMICIDE", "HUMAN TRAFFICKING", "KIDNAPPING",
                 "NARCOTICS", "OFFENSE INVOLVING CHILDREN", "PROSTITUTION", "BURGLARY"]:
        return "FELONY"
    
    elif value in ["ARSON", "ASSAULT", "BURGLARY","DECEPTIVE PRACTICE","MOTOR VEHICLE THEFT", "INTIMIDATION",
               "ROBBERY","SEX OFFENSE", "THEFT", "CRIMINAL TRESPASS"]:
        return "MISDEMEANOR"
    
    elif value in  ["CONCEALED CARRY LICENSE VIOLATION","OTHER NARCOTIC VIOLATION", "PUBLIC INDECENCY",
               "STALKING", "OBSCENITY", "LIQUOR LAW VIOLATION", "PUBLIC PEACE VIOLATION",
                "INTERFERENCE WITH PUBLIC OFFICER", "OTHER OFFENSE", "WEAPONS VIOLATION", "BATTERY"]:
        return "INFRACTION"

    elif value in ["RITUALISM", "GAMBLING"]:
        return "OTHER"
    
crimes["Crime_Type"] = list(map(translation,crimes["Primary Type"]))

In [126]:
crimes.head()

,Case Number,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,date,time,Weekday,Crime_Type
ID,,,,,,,,,,,,,,,,,
13230310,JG447808,2018-01-01,041XX N ASHLAND AVE,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,0,0,1912,19,47.0,6.0,2018,2018-01-01,00:00:00,0,MISDEMEANOR
13235163,JG453401,2018-01-01,027XX N MANGO AVE,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,0,1,2514,25,30.0,19.0,2018,2018-01-01,00:00:00,0,FELONY
11848137,JC457303,2018-01-01,081XX S EBERHART AVE,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,RESIDENCE,1,0,631,6,6.0,44.0,2018,2018-01-01,00:00:00,0,FELONY
11768997,JC362338,2018-01-01,001XX N ASHLAND AVE,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,RESIDENCE,0,1,1224,12,27.0,28.0,2018,2018-01-01,00:00:00,0,MISDEMEANOR
12057588,JD240441,2018-01-01,048XX N AVERS AVE,OFFENSE INVOLVING CHILDREN,CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,APARTMENT,0,1,1712,17,35.0,14.0,2018,2018-01-01,00:00:00,0,FELONY
